In [1]:
### Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import gc


In [2]:
# get working directory and remove last folder
# TODO: make this more robust
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print('Working Directory: ', os.getcwd())

Working Directory:  n:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance


In [15]:
dtypes = {
    'level': np.uint8,
    "level_group": "category",
    'event_name': np.uint8,
    'name': np.uint8,
    'fqid': np.uint8,
    'room_fqid': np.uint8,           
    "text_fqid": np.uint8,
    'fullscreen': np.uint8,
    'hq': np.uint8,
    'music': np.uint8,
    'hover_duration_mean': np.float32,
    'difference_clicks_mean': np.float32,
    'elapsed_time_std': np.float32,
    'page_std': np.float32,
    'room_coor_x_std': np.float32,
    'room_coor_y_std': np.float32,
    'screen_coor_x_std': np.float32,
    'screen_coor_y_std': np.float32,
    'hover_duration_std': np.float32,
    'difference_clicks_std': np.float32,
    'index_sum_of_actions': np.float32,
    'difference_clicks_max': np.float32,
    'elapsed_time_max': np.float32,
    'clicks_per_second': np.float32}

dataset_df = pd.read_csv("data/processed/dataset_df_level.csv", dtype=dtypes)

#load the label dataset
labels = pd.read_csv("data/processed/labels.csv")
dataset_df_0_4 = pd.read_csv("data/processed/df_0_4.csv", dtype=dtypes)

In [4]:
def split_dataset(dataset, test_ratio=0.20):
    USER_LIST = dataset_df.index.unique()
    split = int(len(USER_LIST) * (1 - 0.20))
    return dataset.loc[USER_LIST[:split]], dataset.loc[USER_LIST[split:]]

train_x, valid_x = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_x), len(valid_x)))

433073 examples in training, 108269 examples in testing.


In [5]:
# Fetch the unique list of user sessions in the validation dataset. We assigned 
# `session_id` as the index of our feature engineered dataset. Hence fetching 
# the unique values in the index column will give us a list of users in the 
# validation set.
VALID_USER_LIST = valid_x.index.unique()

# Create a dataframe for storing the predictions of each question for all users
# in the validation set.
# For this, the required size of the data frame is: 
# (no: of users in validation set  x no of questions).
# We will initialize all the predicted values in the data frame to zero.
# The dataframe's index column is the user `session_id`s. 
prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)

# Create an empty dictionary to store the models created for each question.
models = {}

# Create an empty dictionary to store the evaluation score for each question.
evaluation_dict ={}

In [8]:
#get labels and data together
# Iterate through questions 1 to 18 to train models for each question, evaluate
# the trained model and store the predicted values.
for q_no in range(1,19):

    # Select level group for the question based on the q_no.
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    print("### q_no", q_no, "grp", grp)
    
        
    # Filter the rows in the datasets based on the selected level group. 
    train_df = train_x.loc[train_x.level_group == grp]
    train_users = train_df.index.values
    valid_df = valid_x.loc[valid_x.level_group == grp]
    valid_users = valid_df.index.values

    # Select the labels for the related q_no.
    train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
    valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]

    # Add the label to the filtered datasets.
    train_df["correct"] = train_labels["correct"]
    valid_df["correct"] = valid_labels["correct"]

### q_no 1 grp 0-4


KeyError: "None of [Int64Index([     0,      1,      2,      3,      4,     23,     24,     25,\n                26,     27,\n            ...\n            433030, 433031, 433032, 433033, 433034, 433053, 433054, 433055,\n            433056, 433057],\n           dtype='int64', name='session', length=94250)] are in the [index]"

In [39]:
import pandas as pd

def combine_rows(df):
    """
    Combines every 5 rows of a Pandas DataFrame into a new DataFrame, with each row containing the combined values from the 5 rows.
    
    Args:
        df (pandas.DataFrame): The DataFrame to combine.
        
    Returns:
        pandas.DataFrame: A new DataFrame containing one row for every 5 rows of the input DataFrame, with each row containing the combined values from the 5 rows.
    """
    # Determine the number of rows and columns in the input DataFrame
    num_rows, num_cols = df.shape

    # Determine the number of new rows in the output DataFrame
    num_new_rows = num_rows // 5

    # Create an empty list to store the new rows
    new_rows = []

    # Loop through every 5 rows in the input DataFrame and concatenate them into a new row
    for i in range(0, num_rows, 5):
        rows = df.iloc[i:i+5]
        new_row = rows.values.flatten()
        new_rows.append(new_row)

    # Create a new dataframe with the correct number of columns
    num_new_cols = num_cols * 5
    col_names = [f"{col}_{i}" for i in range(1, 6) for col in df.columns]
    new_df = pd.DataFrame(columns=col_names)

    # Add the new rows to the new dataframe
    for i in range(num_new_rows):
        start_index = i * 5
        end_index = (i + 1) * 5
        new_df.loc[i] = new_rows[start_index:end_index]

    return new_df


In [40]:
# Example usage
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                   'B': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   'C': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]})
new_df = combine_rows(dataset_df_0_4)
print(new_df)

ValueError: cannot set a row with mismatched columns